# Chapter 9. ESNの内部結合の学習

この章では、前章の手法を発展させ、ESNの内部結合をオンライン学習で調整する手法を扱います。

**注意:** この章のコードはGoogle Colaboratory上では動作しません。READMEを参照の上、ローカル環境を構築し、その上で実行してください。

## 前書き

通常、ESNを用いたRCではESNの内部結合は固定されます。
一方で内部結合は、前回議論されたように、閉ループの一種としてみなせます。
例えば以下のESNを考えます。

$$
\begin{align*}
x[k+1] &= \tanh\left(\rho W^\mathrm{rec} x[k] + W^\mathrm{in} u[k+1]+ W^\mathrm{feed} y[k] \right)\\
y[k] &= W^\mathrm{out}x[k]
,\end{align*}
$$

この時、 $y[k]$ を代入すると以下の式になります。

$$
\begin{align*}
x[k+1] &= \tanh\left(\rho W^\mathrm{rec} x[k] + W^\mathrm{in} u[k+1]+ W^\mathrm{feed}W^\mathrm{out}x[k] \right)\\
&= \tanh\left(\left(\rho W^\mathrm{rec} + W^\mathrm{feed}W^\mathrm{out}\right) x[k] + W^\mathrm{in} u[k+1] \right)
.\end{align*}
$$

つまり内部結合が $\rho W^\mathrm{rec} + W^\mathrm{feed}W^\mathrm{out}$ とみなせます。
また逆の操作で、あるESN上の $i$ 番目のノードに結合する前結合 $W^\mathrm{rec}_{i~\cdot}$ 、後結合 $W^\mathrm{rec}_{\cdot~i}$ をそれぞれ $W^\mathrm{out}$、$W^\mathrm{feed}$ として内部結合を閉ループとして外部化できます。
あとは前章の閉ループの学習プロセスで調整すれば内部結合を調整できます。
ここで新たにどのように目標時系列を指定するか問題となります。

今回紹介する **生得的学習 (innate training)**<sup>[1]</sup>ならびに **full-FORCE**<sup>[2]</sup>はいずれも別のESNを用いて目標時系列を生成し、FORCE学習<sup>[3]</sup>によって学習を達成します。
以下その概要を説明します。

### 生得的学習

**生得的学習**<sup>[1]</sup> はR. Lajeらによって提案されたESNの内部結合を調整するオンライン学習法の一種です。
生得的学習の特徴的な点はその目標時系列の指定方法です。
特にカオスESNを用意し、ある入力が与えられた後のカオスESNの内部状態の時系列を、別のESNで一定期間再現的に生成するように内部結合が学習されます。
またこの目標となる軌道は **生得的軌道 (innate trajectory)** と呼ばれます。

生得的学習の学習プロセスは、事前学習と事後学習の2段階に分けられます。
事前学習では、FORCE学習<sup>[3]</sup>を用いて内部結合が調整されます。
事後学習は、通常のRCと同様に、事前学習の後に行われます。
すなわち開ループ系で特定の軌道を出力するようにリードアウト層が学習されます。
このときのコスト関数 $C(W^\mathrm{rec})$ は以下のとおり表されます。

$$
\begin{align*}
C(W^\mathrm{rec}):= \sum_{k \in T} \|x^\mathrm{target}[k] - x[k]\|^2
,\end{align*}
$$

$x^\mathrm{target}[k]$ は生得的軌道、$T$ はある時間の範囲です。
生得的学習ではある初期値 $x^\mathrm{target}[t_0]$ とある入力に対する時系列をターゲットとして採用するケースが多いです。

生得的学習の興味深い点は、カオス性が事前学習後にも完全に抑圧されない点にあります。
即ち全体としては系がカオス的にも関わらず、局所的にESPが回復し一定期間高次元な複雑な軌道が再現的に生成されるのです。
またデモンストレーションで後ほど示されますが、その高次元なカオス軌道は高い表現能力を有し、様々な軌道を設計できます。
またこの高次元カオスは、局所的なパターンのみならず、大域的なパターン間の切り替え則を埋め込み、カオス的遍歴上の軌道を設計できます<sup>[4]</sup>（[論文のコード参照](https://github.com/katsuma-inoue/designing_chaotic_itinerancy_demo)）。

### Full-FORCE

B. DePasqualeらによって提案された**Full FORCE**<sup>[2]</sup>もまた、ESNの内部状態を調整するオンライン学習法です。
生得的学習とは異なり、カオスESNとその軌道を目標時系列としては使用しません。
代わりに以下の式で表される、ある入力 $u^\mathrm{embed}[k]$ が与えられた際の非カオスESN（ $\rho<1$ ）の軌道を目標に据えます。

$$
\begin{align*}
x^\mathrm{target}[k+1] &= (1-a)x^\mathrm{target}[k] + a\tanh\left(\rho W^\mathrm{rec} x^\mathrm{target}[k] + W^\mathrm{in} u^\mathrm{in}[k]+ W^\mathrm{embed} u^\mathrm{embed}[k] \right)
,\end{align*}
$$

$u^\mathrm{embed}[k]$ は埋め込み対象の追加の入力で、 $u^\mathrm{in}[k]$ の関数であるものを用います。
full-FORCEでは、この $u^\mathrm{embed}[k]$ **なしに** ESNへの埋め込みを目指します。
つまりこのときのコスト関数 $C(W^\mathrm{rec})$ は以下の式で表されます。

$$
\begin{align*}
C(W^\mathrm{rec}):= \sum_{k \in T} \|x^\mathrm{target}[k] + W^\mathrm{embed}u^\mathrm{embed}[k] - x[k]\|^2
,\end{align*}
$$

Full-FORCE学習でも事後学習は存在し、追加のリードアウト層が調整されます。
full-FORCEの特徴として、この $W^\mathrm{embed} u^\mathrm{embed}[k]$ の埋め込みにより、FORCE学習単体より複雑な時系列の設計が可能になる点が挙げられます。
このようにfull-FORCEは、内部結合の調整により時系列の設計性を向上させる手法といえます。

## 演習問題と実演

前回と同様、各種ライブラリおよび実装済みの関数の`import`を行うために次のセルを実行してください。
なお内部実装を再確認するには、`import inspect`以下の行をコメントアウトするか`...?? / ??...`を使用してください。

In [ ]:
import json
import os
import sys
from collections import defaultdict

import joblib
import numpy as np
from IPython.display import clear_output, display
from ipywidgets import Output

if "google.colab" in sys.modules:
    from google.colab import drive  # type: ignore

    if False:  # Set to True if you want to use Google Drive and save your work there.
        drive.mount("/content/gdrive")
        %cd /content/gdrive/My Drive/rc-bootcamp/
        # NOTE: Change it to your own path if you put the zip file elsewhere.
        # e.g., %cd /content/gdrive/My Drive/[PATH_TO_EXTRACT]/rc-bootcamp/
    else:
        pass
        %cd /content/
        !git clone --branch ja_sol https://github.com/rc-bootcamp/rc-bootcamp.git
        %cd /content/rc-bootcamp/
else:
    sys.path.append(".")

from utils.interface import InteractiveViewer
from utils.interpolate import interp1d
from utils.reservoir import ESN, RidgeReadout, rls_update
from utils.style_config import plt
from utils.tester import load_from_chapter_name
from utils.tqdm import tqdm, trange
from utils.viewer import show_innate_error, show_innate_record

test_func, show_solution = load_from_chapter_name("09_internal_optimization")


# Uncomment it to see the implementations.
# import inspect
# print(inspect.getsource(Linear))
# print(inspect.getsource(RidgeReadout))
# print(inspect.getsource(ESN))
# print(inspect.getsource(rls_update))

# Or just use ??.../...?? (uncomment the following lines).
# Linear??
# RidgeReadout??
# ESN??
# rls_update??

### 事前学習の実装

まずは 生得的学習の事前学習の実装から始めましょう。
先述のとおり生得的学習は、各ノードの前結合を線形閉ループと見なし、FORCE学習を用いて内部結合を調整する手法です。
次の式は、$i$ 番目の前結合 ($1\leq i \leq N$) に対する FORCE 学習に基づく重み更新則を示しています。

$$
\begin{align*}
k^{i} &= P^{i} x^{i} \\
g^{i} &= \frac{1}{1+{x^{i}}^\top k^{i}} \\
\Delta P^{i} &= g^{i}{k}^{i}{k^{i}}^\top \\
\Delta W^\mathrm{rec}_{i~\cdot} &= g^{i} (\hat{x}^{i} - {x}^{i}) {k}^{i} \\
P^{i} &\leftarrow P^{i} - \Delta P^{i} \\
W^\mathrm{rec}_{i~\cdot}  &\leftarrow W^\mathrm{rec}_{i~\cdot}  + \Delta W^\mathrm{rec}_{i~\cdot}
,\end{align*}
$$

ここで、$P^{i} \in \mathbb{R}^{N^{i}\times N^{i}}$ は ${I}/{\alpha}$ として初期化された正定値行列であり、$\hat{x}$ は 生得的軌道です。
RLSアルゴリズムに基づくFORCE学習を使用しているため、前章で実装した `rls_update` を再利用できます。
ESNの内部結合が疎（`sparse < 1.0`; $p<1.0$ ）であるため、$N^{i}$ は $N$ よりも小さい値を取り ( $N^{i} \approx p N$ )、計算量が $O(N\times N^2)$ と比較して大幅に小さい点に注意してください（$O(p^2 N^3)$）。

Q1.1.

以下の穴埋めを実装し、 `InnateESN.__init__` と `InnateESN.train` を完成させよ。

- `InnateESN.train`
  - Argument(s)
    - `x_target`: `np.ndarray`
      - innate trajectory $\hat{x}[k]$ ($\in \mathbb{R}^{\cdots \times N}$)
    - `x_now`: `np.ndarray`
      - $x[k]$ ($\in \mathbb{R}^{\cdots \times N}$)
  - Return(s)
    - `self.w_net`: `np.ndarray`
      - $\Delta W^\mathrm{rec}$

  - Operation(s)
    - `rls_update`を用いた`InnateESN.P`の更新（ $I/\lambda$ で初期化）
    - `InnateESN.w_net` の更新

<details><summary>tips</summary>

- [`np.nonzero`](https://numpy.org/doc/stable/reference/generated/numpy.nonzero.html)

</details>

In [ ]:
class InnateESN(ESN):
    def __init__(self, *args, lmbd=1.0, **kwargs):
        """
        Tunable ESN [Laje, R., & Buonomano, D. V. (2013). Nature neuroscience, 16(7), 925-933.]

        Args:
            alpha (float, optional): regularization parameter for RLS algorithm. Defaults to 1.0.
        """
        super(InnateESN, self).__init__(*args, **kwargs)
        self.w_pre = {}
        self.P = {}
        for post in range(self.dim):
            non_zeros = self.weight[post].nonzero()[0]
            self.w_pre[post] = non_zeros
            self.P[post] = np.eye(len(self.w_pre[post])) / lmbd  # TODO Initialize P matrix for RLS.

    def train(self, x_target, x_now=None, node_list=None):
        """
        Update the internal weight by RLS algorithm

        Args:
            x_target (np.ndarray): State(s) on an inante trajectory.
            x_now (np.ndarray, optional): Current state(s). Defaults to None (use self.x).
            node_list (list, slice, optional): Tuned nodes. Defaults to None (train all nodes).
        """
        if x_now is None:
            x_now = np.asarray(self.x)
        if node_list is None:
            node_list = range(self.dim)
        for xt, xn in zip(x_target.reshape(-1, self.dim), x_now.reshape(-1, self.dim), strict=False):
            es = xt[node_list] - xn[node_list]
            for node_id, e in zip(node_list, es, strict=False):
                x = xn[self.w_pre[node_id]]  # TODO Use self.w_pre (hint: `x = xn[...]`).
                P = self.P[node_id]  # TODO Get P matrix for the node (hint: `P = self.P[...]`).
                g, k, P_new = rls_update(P, x)  # TODO Use `rls_update`.
                dw = g * np.outer(e, k)  # TODO Calculate dw (hint: use `g`, `e`, and `k`).
                self.P[node_id] = P_new
                self.weight[node_id, self.w_pre[node_id]] += dw[0]
        return self.weight

    def to_pickle(self, file_name):
        os.makedirs(os.path.dirname(file_name), exist_ok=True)
        P, w_pre = self.P, self.w_pre
        self.P, self.w_pre = {}, {}
        with open(file_name, mode="wb") as f:
            joblib.dump(self, f, compress=True)
        self.P, self.w_pre = P, w_pre

    @staticmethod
    def read_pickle(file_name):
        with open(file_name, mode="rb") as f:
            module = joblib.load(f)
        return module


def solution(dim, seed, x_target, x_now, node_list):
    # DO NOT CHANGE HERE.
    net = InnateESN(dim, seed=seed, node_list=node_list)
    net.train(x_target=x_target, x_now=x_now, node_list=node_list)
    return net.weight


test_func(solution, "01_01")
# show_solution("01_01", "InnateESN")  # Uncomment it to see the solution.

Q1.2.

生得的学習の事前学習を実装する `emulate_innate` を実装せよ。
ただし $k \in [t_\mathrm{b}, t_\mathrm{e}) \land k \equiv 0~ (\mathrm{mod}~t_\mathrm{every}) $ のときに `InnateESN.train` によって $W^\mathrm{rec}$ を更新、それ以外のときは何もしない。

- `emulate_innate`
  - Argument(s):
    - `ts`: `list | np.ndarray`
    - `net`: `InnateESN`
    - `f_in`: `Callable`
      - $f^\text{in}(t)$
    - `innate_range`: `tuple(int, int)`
      - $[t_\mathrm{b}, t_\mathrm{e})$
    - `innate_node`: `list | slice`
    - `innate_every`: `int`
      - $t_\mathrm{every}$
    - `innate_func`: `Callable`
      - $\hat{x}: T \to \mathbb{R}^{\cdots \times N}$
  - Return(s):
    - `record`: `dict`
      - `'t'`: `np.ndarray`
        - $[t_0, t_1,~\ldots,~t_{T-1}]$
      - `'x'`: `np.ndarray`
        - $[x[t_0], x[t_1],~\ldots,~x[t_{T-1}]]$

In [ ]:
def emulate_innate(
    ts,
    net,
    f_in=None,
    innate_range=None,
    innate_func=None,
    innate_node=None,
    innate_every=2,
    prefix="",
    leave=True,
    display=True,
):
    record = {}
    record["t"] = np.zeros(len(ts), dtype=int)
    record["x"] = np.zeros((len(ts), *net.x.shape))
    pbar = tqdm(ts, leave=leave, display=display)
    for cnt, t in enumerate(pbar):
        pbar.set_description("{}t={:.0f}".format(prefix, t))
        u_in = np.zeros_like(net.x)
        if f_in is not None:
            u_in += f_in(t)
        net.step(u_in)
        record["t"][cnt] = t
        record["x"][cnt] = net.x
        if (innate_range is not None) and (innate_range[0] <= t < innate_range[1]):
            if cnt % innate_every == 0:
                # TODO Use `net.train` and specify `node_list=innate_node`.
                x_target = innate_func(t)
                net.train(x_target, node_list=innate_node)
                # end of TODO
    return record


def solution(ts, dim, seed, **kwargs):
    # DO NOT CHANGE HERE.
    net = InnateESN(dim, seed=seed)
    record = emulate_innate(ts, net, **kwargs)
    return record["x"]


test_func(solution, "01_02")
# show_solution("01_02", "emulate_innate")  # Uncomment it to see the solution.

`emulate_innate` が準備できたら以下のセルを実行してください。

- `create_target`: 生得的軌道の生成
- `train_network`: 事前学習の実行
- `eval_newtork`: 事前学習の評価
- `eval_error`: MSEならびにNRMSEの評価

In [ ]:
def create_target(record_range, net, f_in, rnd=None):
    rnd = rnd if rnd else np.random.default_rng()
    net.x = rnd.uniform(low=-1, high=1, size=(f_in.dim, net.dim))
    record = emulate_innate(range(*record_range), net, f_in=f_in, prefix="sample ")
    return record


def train_network(record_range, net, f_in, rec_target, innate_range, innate_node, innate_every, rnd=None):
    rnd = rnd if rnd else np.random.default_rng()
    innate_func = interp1d(rec_target["x"], x=rec_target["t"], kind=1, axis=0)
    net.x = rnd.uniform(low=-1, high=1, size=(f_in.dim, net.dim))
    record = emulate_innate(
        range(*record_range),
        net,
        f_in=f_in,
        prefix="train ",
        innate_func=innate_func,
        innate_node=innate_node,
        innate_range=innate_range,
        innate_every=innate_every,
    )
    return record


def eval_network(record_range, net, f_in, eval_num=5, rnd=None):
    rnd = rnd if rnd else np.random.default_rng()
    net.x = rnd.uniform(low=-1, high=1, size=(eval_num, f_in.dim, net.dim))
    record = emulate_innate(range(*record_range), net, f_in=f_in, prefix="eval ")
    return record


def eval_error(rec_eval, rec_target, innate_range, innate_node):
    begin_id = int(innate_range[0] - rec_eval["t"][0])
    end_id = int(innate_range[1] - rec_eval["t"][0])
    diff = rec_eval["x"] - rec_target["x"][:, None]  # -> [Time_steps, Eval_num, w_pulse_num, Net_dim]
    norm = (diff[begin_id:end_id, ..., innate_node] ** 2).sum(axis=-1)  # -> [T, E, W]
    mse = norm.mean(axis=0)  # -> [E, W]

    var = (rec_target["x"][begin_id:end_id, ..., innate_node] ** 2).sum(axis=(0, 2))  # -> [W]
    nrmse = (norm.sum(axis=0) / var) ** 0.5  # -> [E, W]
    return {"mse": mse, "nrmse": nrmse}

### 事前学習の実行

ここからデモンストレーションに移ります。
まず実験パラメータを指定します。
また事前学習に非常に時間がかかるため、出力フォルダ `save_dir` （デフォルトでは`./output`）を作成し、実験条件と結果を保存します。

In [ ]:
save_dir = "./result"
os.makedirs(save_dir, exist_ok=True)

seed = 1234

w_pulse_num = 1
dim, a, sr, p = 1000, 0.1, 1.6, 0.2
pulse_amp, pulse_period = 10.0, 50

alpha = 1.0
innate_epoch = 20
innate_period, innate_rate, innate_every = 3000, 0.1, 4
washout_period, record_period = 1000, 5000

rnd = np.random.default_rng(seed)
net = InnateESN(dim, a=a, sr=sr, p=p, alpha=alpha, rnd=rnd)
w_pulse = rnd.uniform(size=(w_pulse_num + 2, dim), low=-1.0, high=1.0)

innate_range = [0, innate_period]
innate_node = list(range(0, int(innate_rate * net.dim)))
record_range = [-washout_period, record_period]
pulse_range = [-pulse_period, 0]

plot_range = list(range(5))


def f_in(t):
    if -pulse_period <= t < 0:
        return pulse_amp * w_pulse[:w_pulse_num]
    else:
        return 0.0


f_in.dim = w_pulse_num

with open(f"{save_dir}/params.json", mode="w") as f:
    json.dump(
        {
            "pulse_amp": pulse_amp,
            "pulse_period": pulse_period,
            "innate_period": innate_period,
            "innate_rate": innate_rate,
            "innate_every": innate_every,
        },
        f,
        indent=4,
    )

np.save(f"{save_dir}/w_pulse.npy", w_pulse)
net.to_pickle(f"{save_dir}/net_init.pkl")

まずは生得的軌道 $\hat{x}[k]$ を生成します。

In [ ]:
rec_target = create_target(record_range, net, f_in, rnd=rnd)
fig_target = show_innate_record(
    rec_target,
    plot_range,
    lw=1.5,
    color="k",
    title="innate trajectory",
    pulse_range=pulse_range,
    innate_range=innate_range,
)
np.savez_compressed(f"{save_dir}/rec_target.npz", *rec_target)
fig_target.savefig(f"{save_dir}/rec_target.png", dpi=200)

灰色の領域は、パルス入力が与えられた期間を表します。
ピンク色の領域は、事前学習期間 $[t_\mathrm{b}, t_\mathrm{e})$ を表します。

In [ ]:
def eval_network(record_range, net, f_in, eval_num=5, rnd=None):  # noqa: F811
    rnd = rnd if rnd else np.random.default_rng()
    net.x = rnd.uniform(low=-1, high=1, size=(eval_num, f_in.dim, net.dim))
    record = emulate_innate(range(*record_range), net, f_in=f_in, prefix="eval ")
    return record


rec_init = eval_network(record_range, net, f_in, rnd=rnd)
fig = show_innate_record(
    rec_target,
    plot_range,
    lw=1.5,
    ls=":",
    clear=True,
    color="k",
    pulse_range=pulse_range,
    innate_range=innate_range,
)
fig = show_innate_record(
    rec_init,
    plot_range,
    lw=0.5,
    ls="-",
    fig=fig,
    cmap=plt.get_cmap("tab10"),
    title="eval (before pre-training)",
)
fig.savefig(f"{save_dir}/eval/init.png", dpi=200)

この図より、生得的軌道 (点線) をESNが現時点では再現的に生成できない様子が読み取れます。

以下のセルは事前学習を実行します。
20〜30分はかかりますので、終了するまでお待ちください。

In [ ]:
record = {}
best_score = np.array(np.inf)
figs = defaultdict(lambda: None)

pbar = trange(innate_epoch)
out_tqdm, out_figure = Output(), Output()
display(out_tqdm)
display(out_figure)
for epoch in pbar:
    pbar.set_description("best:{:.2e}".format(best_score))
    with out_tqdm:
        clear_output(wait=True)
        # Training phase
        rec_train = train_network(
            record_range,
            net,
            f_in,
            rec_target,
            innate_range,
            innate_node,
            innate_every,
            rnd=rnd,
        )
        figs["train"] = show_innate_record(
            rec_target,
            plot_range,
            lw=1.5,
            ls=":",
            clear=True,
            color="black",
            fig=figs["train"],
            pulse_range=pulse_range,
            innate_range=innate_range,
        )
        figs["train"] = show_innate_record(
            rec_train,
            plot_range,
            lw=1.0,
            ls="-",
            fig=figs["train"],
            cmap=plt.get_cmap("tab10"),
            title=f"train (epoch #{epoch})",
        )
        figs["train"].savefig("{}/train/{:03d}.png".format(save_dir, epoch), dpi=200)

        # Evaluation phase
        rec_eval = eval_network(record_range, net, f_in, rnd=rnd)
        figs["eval"] = show_innate_record(
            rec_target,
            plot_range,
            lw=1.5,
            ls=":",
            clear=True,
            color="black",
            fig=figs["eval"],
            pulse_range=pulse_range,
            innate_range=innate_range,
        )
        figs["eval"] = show_innate_record(
            rec_eval,
            plot_range,
            lw=0.5,
            ls="-",
            fig=figs["eval"],
            cmap=plt.get_cmap("tab10"),
            title=f"eval (epoch #{epoch})",
        )
        figs["eval"].savefig("{}/eval/{:03d}.png".format(save_dir, epoch), dpi=200)

        # Record evaluation error
        rec = eval_error(rec_eval, rec_target, innate_range, innate_node)
        best_score = min(best_score, rec["nrmse"].sum())
        rec["best"] = best_score

        for key, val in rec.items():
            if key not in record:
                record[key] = np.zeros((innate_epoch, *val.shape))
            record[key][epoch] = val

    with out_figure:
        clear_output(wait=True)
        figs["nrmse"] = show_innate_error(record["nrmse"][: epoch + 1], fig=figs["nrmse"])
        figs["nrmse"].savefig(f"{save_dir}/nrmse.png", dpi=200)
        for _name, fig in figs.items():
            size = fig.get_size_inches()
            fig.set_size_inches(8, 3)
            display(fig)
            fig.set_size_inches(size)

net.to_pickle(f"{save_dir}/net_term.pkl")
np.savez_compressed(f"{save_dir}/record.npz", **record)

for _name, fig in figs.items():
    fig.close()

徐々に、異なる初期値にもかかわらず、複雑な高次元カオスを再現的に生成できる様子が見て取れると思います。
`{save_dir}/eval` 内に保存されている図を確認してください。

Q1.3. (Advanced)

- ノード数を変化させ、MSEの違いを観察せよ。
- より長い `innate_period` を試し、事前学習の性能を比較せよ。
- `w_pulse_num` を増やし、複数の入力に対して事前学習せよ。

### 事後学習の実装

事前学習したESNを用いて、リードアウト層を学習しましょう。
以下のセルは`{load_dir}/net_term.pkl`に保存されたESNと、`{load_dir}/params.json` に保存されたパラメータファイルを読み込みます。
事前学習済みの場合は前のセルをスキップして、ここから開始可能です。

In [ ]:
load_dir = "./result"

net_init = InnateESN.read_pickle(f"{load_dir}/net_init.pkl")
net_term = InnateESN.read_pickle(f"{load_dir}/net_term.pkl")
w_pulse = np.load(f"{load_dir}/w_pulse.npy")
with open(f"{load_dir}/params.json", mode="r") as f:
    params = json.load(f)

pulse_amp = params["pulse_amp"]
pulse_period = params["pulse_period"]
innate_period = params["innate_period"]

以下のセルは、事前学習前と後のESNの軌道を比較します。

In [ ]:
seed = 1234
w_pulse_num = 1
washout_period, record_period = 1000, 10000
record_range = [-washout_period, record_period]
pulse_range = [-pulse_period, 0]
innate_range = [0, innate_period]
plot_range = list(range(5))

rnd = np.random.default_rng(seed)


def f_in(t):
    if -pulse_period <= t < 0:
        return pulse_amp * w_pulse[:w_pulse_num]
    else:
        return 0.0


f_in.dim = w_pulse_num

rec_sample_init = eval_network(record_range, net_init, f_in, rnd=rnd)
fig = show_innate_record(
    rec_sample_init,
    plot_range,
    lw=1.0,
    title="initial",
    pulse_range=pulse_range,
    innate_range=innate_range,
)

rec_sample_term = eval_network(record_range, net_term, f_in, rnd=rnd)
fig = show_innate_record(
    rec_sample_term,
    plot_range,
    lw=1.0,
    title="pre-trained",
    pulse_range=pulse_range,
    innate_range=innate_range,
)

次に目標となるリードアウト層の出力時系列を用意しましょう。
`./data/09_internal_optimization`以下には`abc.csv`と`star.csv`が用意されています。
今回は`abc.csv`を使用してみましょう。

In [ ]:
data = np.loadtxt("./data/09_internal_optimization/abc.csv", delimiter=",")

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(data[:, 0], data[:, 1])
ax.set_aspect("equal")

事前学習したESNは、入力後 $t\in[0, t_\mathrm{innate})$ の期間再現的にinnate trajectoryを生成します。
これをリードアウト層を用いて線型回帰により対応付けましょう。

In [ ]:
ts, xs = rec_sample_term["t"], rec_sample_term["x"]

x_train = xs[washout_period : washout_period + innate_period, :, 0, :]

target_func = interp1d(data, kind=1, axis=0)
ds = target_func(np.linspace(0, 1, innate_period))
ds = np.broadcast_to(ds[:, None, :], (*x_train.shape[:-1], ds.shape[-1]))

w_out = RidgeReadout(net_term.dim, 2, lmbd=1e-2)
weight, bias = w_out.train(x_train.reshape(-1, x_train.shape[-1]), ds.reshape(-1, ds.shape[-1]))

`InteractiveView` は動的に入力を与えられるツールです。
以下のセルを実行し、パルス入力後に適切に目標軌道が出力されるか確認してください。

In [ ]:
%matplotlib widget

import copy

try:
    del viewer  # type: ignore
except NameError:
    pass

net_term_cp = copy.deepcopy(net_term)

net_term_cp.x = rnd.uniform(-1, 1, net_term_cp.dim)
viewer = InteractiveViewer(
    net_term_cp,
    w_out,
    w_pulse,
    pulse_amp,
    pulse_period,
    plot_num=5,
    input_num=w_pulse_num,
    max_time_steps=10000,
    cmap="Greens",
)

viewer.view()

Q2.1. (Advanced)

- `InteractiveView` の引数 `input_num` の数を増やすと入力の種類を増やせる。
ただ現時点では学習されていないため、意味のない出力しか得られない。
入力の数を2に増やし（`w_pulse_num=2`）、別の入力に対しては「星」（`star.csv`）を出力するように事前学習と事後学習を行え (僅かな変更で実装できる) 。

Q2.2. (Advanced)

- Innate trainingとほとんどコードを改変せずにFull FORCEを実装できる。[2]を参考に、`InnateESN`を継承した`FullFORCEESN`を実装せよ。

## 参考文献

[1] Laje, R., & Buonomano, D. V. (2013). *Robust timing and motor patterns by taming chaos in recurrent neural networks*. Nature Neuroscience, 16(7), 925–933. https://doi.org/10.1038/nn.3405

[2] DePasquale, B., Cueva, C. J., Rajan, K., Escola, G. S., & Abbott, L. F. (2018). *full-FORCE: A target-based method for training recurrent networks*. PLOS ONE, 13(2), e0191527. https://doi.org/10.1371/journal.pone.0191527

[3] Sussillo, D., & Abbott, L. F. (2009). *Generating Coherent Patterns of Activity from Chaotic Neural Networks*. Neuron, 63(4), 544–557. https://doi.org/10.1016/j.neuron.2009.07.018

[4] Inoue, K., Nakajima, K., & Kuniyoshi, Y. (2020). *Designing spontaneous behavioral switching via chaotic itinerancy*. Science Advances, 6(46), eabb3989. https://doi.org/10.1126/sciadv.abb3989